# Pokročilé dolování z databází - Průběžná zpráva
### Autoři: Lukáš Vala, Dan Balarin

# Použitá data
V tomto návodu budeme pracovat s dvěma datasety:
 - [Adult](https://archive-beta.ics.uci.edu/dataset/2/adult) - Obsahuje data z roku 1994. Obsahuje informace o lidech z celého světa a k tomu navíc, zda daná osoba vydělává více či méně než 50000 dolarů za rok. Můžeme si tedy pokládat otázky, jak ovlivňuje plat národnost, rasa, pohlaví a podobně.
    - Počet řádků: 48842
    - Počet slouců: 14
 - [GDP in 1994](https://data.worldbank.org/indicator/NY.GDP.PCAP.CD?end=1994&most_recent_year_desc=true&start=1993) - Obsahuje informace o hrubém domácím produktu jednotlivých států v roce 1994. 
    - Počet řádků: 266
    - Počet slouců: 3 - jeden z těchto sloupců bude v tomto návodu smazán

 Oba datasety obsahují sloupec stát - proto je budeme dále v dokumentu spojovat podle tohoto sloupce.

# Vypracování návodu pro zpracování a analýzu dat pomocí systému CleverMiner
## Krok 0 Instalace potřebných knihoven
- Clever Miner: `pip install cleverminer`
- pandas: `pip install pandas`
## Krok 1 Načtení a zpracování dat
Prvním krokem je načíst data pomocí knihovny pandas. Data se v tomto případě nacházejí ve složce data:

In [1]:
import pandas as pd

adult = pd.read_csv ('./data/adult.data', encoding='cp1250', sep=', ')
gdp = pd.read_csv ('./data/gdpStates.txt', encoding='cp1250', sep='\t')

adult.head(5)
gdp.head(5)

/usr/local/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,country,Most Recent Year,GDP per Capita
0,Saudi Arabia,1994,7421.3
1,Qatar,1994,14544.0
2,Ukraine,1994,1012.0
3,Jordan,1994,1414.3
4,Indonesia,1994,912.2


Pomocí parametru `sep` je důležité specifikovat, jaký separátor jednotlivých atributů je v tabulce použit. U datasetu adult je to `, ` a u GDP je to tabulátor. Správnost načtení dat můžeme poté zkontrolovat pomocí výpisu prvních 5 hodnot obou tabulek.

Dále bychom si také měli zkontrolovat, zda pandas správně určila datové typy u jednotlivých sloupců. 
To můžeme zjistit pomocí příkazu `dtypes`:

In [2]:
print(adult.dtypes)
print(gdp.dtypes)

Age                int64
Workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
country           object
income            object
dtype: object
country              object
Most Recent Year      int64
GDP per Capita      float64
dtype: object


Pro string vypíše pandas `object`, pro integer `int64` a pro desetinné číslo `float64`

## Krok 2 - Spojení dat
Nyní je třeba spojit 2 tabulky do sebe, abychom mohli analýzu provádět nad datasetem Adult společně s informací o hrubém domácím produktu státu, v němž byl daný člověk narozen. 

Oba datasety obsahují sloupec country. V tabulce `adult` se tento sloupec jmenuje `native-country`, je proto nutné ho přejmenovat přímo v datasetu na `country`, aby se shodoval s názvem sloupce ve druhém datasetu. Dále je potřeba upravit jednotlivé hodnoty v jedné z tabulek, protože např. hodnota `United States` v tabulce adult je původně zapsána v gdp tabulce jako `United-States`. Použijeme tedy funkci definovanou v ukázce pod tímto textem a následně v datech GDP najdeme jednotlivé hodnoty a nahradíme je správnými hodnotami z tabulky adult (použita funkce najít a nahradit ve VS-code :) ).

In [3]:
def getUniqueValuesFromColumn(df, col):
    values = df[col].tolist()
    uniqueValues = []
    for i in values:
        if(i not in uniqueValues):
            uniqueValues.append(i)
    return uniqueValues

uniqueCountryValues = getUniqueValuesFromColumn(adult, "country")

Před samotným spojením tabulek ještě odstraníme z GDP tabulky ty státy, které nejsou obsaženy v tabulce adult, aby po spojení nedošlo k vytvoření několika řádků s prázdnými hodnotami (vyplněný by byl pouze stát). Použijeme proto další funkci, do které přidáme dataset, název sloupce a pole hodnot států. Funkce nám pak vráti dataset obsahující pouze řádky se státem, který byl obsažený v poskytnutém poli (uniqueCountryValues z části před touto).

In [4]:
def filter_rows_by_values(df, col, values):
    return df[df[col].isin(values)]

gdp = filter_rows_by_values(gdp, "country", uniqueCountryValues)

Poté již spojíme datasety a vznikne nám nový, který obsahuje všechny atributy z datasetu adult a navíc hrubý domácí produkt státu, ze kterého daná osoba pochází:

In [5]:
adultWithGdp = pd.merge(adult, gdp, on='country', how='outer')

Nyní můžeme vytvořit nové atributy v datasetu podle našich potřeb. Následující kód vytvoří nový sloupec `AgeStatus` a připíše hodnotu "Young" lidem mladším než 30, "Middle aged" lidem mezi 30 a 60 a "Old" lidem starším než 60.

In [6]:
adultWithGdp['AgeStatus'] = adultWithGdp.apply(lambda row: "Young" if row.Age < 30 else ("Middle aged" if row.Age >= 30 & row.Age < 60 else "Old"), axis=1)

## Krok 3 - <a href="https://lispminer.vse.cz/wiki/doku.php?id=mft:start">4ft miner</a>

Reference na [Clever Miner](https://www.cleverminer.org/docs-page.html#section-4)

Analytická procedura 4ft-Miner hledá všechny zajímavé frekvence kombinací hodnot kategorií dvou předzpracovaných atributů, navíc počítané na podmnožinách analyzovaných dat zadaných pomocí booleovských atributů s bohatou syntaxí.
 
U 4ft mineru nastavuje základní parametry:
`conf`: Konfidence - pravděpodobnost, že podmínka bude vyplněna
`base`: Základ - počet vyhovujících případů
`minlen, maxlen`: Maximální a minimální délka podmnožin/sekvencí/cutů

### Příklad 1

V Clever Mineru používáme 4ft miner následovně:

In [7]:
from cleverminer import cleverminer

clm = cleverminer(df=adultWithGdp, proc='4ftMiner',
               quantifiers= {'conf':0.7, 'Base':18000},
               ante ={
                    'attributes':[
                        {'name': 'AgeStatus', 'type': 'subset', 'minlen': 1, 'maxlen': 2}
                    ], 'minlen':1, 'maxlen':1, 'type':'con'},
               succ ={
                    'attributes':[
                        {'name': 'income', 'type': 'subset', 'minlen': 1, 'maxlen': 1}
                    ], 'minlen':1, 'maxlen':1, 'type':'con'}
                )

Cleverminer version  1.0.2  educational version. For production use (bugfixes and keeping compatibility), ask for PRO version.
Cleverminer version 1.0.2. Note: This version is for personal and educational use only. If you need PRO version (support, fixing structures for compactibility in future versions for production deployment, additional development, licensing of commercial use of subroutines used), feel free to ask authors. Most of these functionalities are maintained in best-effort, as soon as this project is at given conditions for free use and rapid development is needed, they cannot be guaranteed.
Starting data preparation ...
Encoding columns into bit-form...
Encoding columns into bit-form...done
Data preparation finished.
Will go for  4ftMiner
Starting to mine rules.
Done. Total verifications : 1, rules 1,control number:0, times: prep 4.043574094772339, processing 0.001699686050415039


V předchozí ukázce jako antecedent nastavujeme vytvořený atribut `AgeStatus` s minimální délkou atributu 1 a maximální délkou 2. To znamená, že clever miner bude vyhledávat podmnožiny z atributu AgeStatus o délce 1 - 2. Jako výsledek tedy můžeme dostat například `AgeStatus(Middle aged)`(délka podmnožiny 1), nebo např. `AgeStatus(Middle aged, Old)`(délka podmnožiny 2), což nám říká, že Age status daného člověka je buď Middle aged nebo Old. Sukcedent jsme nastavili na `income`. Vzhledem k tomu, že atribut `income` může nabývat pouze dvou hodnot, nedává v tomto případě smysl nastavovat maximální délku podmnožiny na 2 - byly by obsaženy všechny hodnoty z `income`.

Konfidenci jsme nastavili na 70% a základ 18 000.

Přepisem do lidsky srozumitelného jazyka toto znamená: Jestliže si vezmeme pro osobu jednu nebo dvě hodnoty z `AgeStatus`(Old/Yound/Middle aged), je v nějakém případě pravděpodobnost, že daná osoba bude vydělávat více či méně než 50K, vyšší než 70% a zároveň je takových osob v datasetu více než 18 000?

Výsledek vypíšeme následovně:

Pomocí následujícího příkazu můžeme zjistit, zda Clever Miner takové pravidlo v datech našel:

In [8]:
clm.print_rulelist()


List of rules:
RULEID BASE  CONF  AAD    Rule
     1 24720 0.759 +0.000 AgeStatus(Middle aged Young) => income(<=50K) | ---



Z výpisu vydíme, že jedno takové pravidlo existuje. Detaily o něm vypíšeme následovně:

In [9]:
clm.print_rule(1)



Rule id : 1

Base : 24720  Relative base : 0.759  CONF : 0.759  AAD : +0.000  BAD : +0.000

Cedents:
  antecedent : AgeStatus(Middle aged Young)
  succcedent : income(<=50K)
  condition  : ---

Fourfold table
    |  S  |  ¬S |
----|-----|-----|
 A  |24720| 7841|
----|-----|-----|
¬A  |    0|    0|
----|-----|-----|



Z tabulky vidíme, že Clever Miner skutečně našel takový vztah mezi `AgeStatus` a `income`. Konkrétně z výsledku můžeme vyvodit toto pravidlo: jestliže je někdo `Middle aged` (30-60), nebo `Young` (< 30), pak je šance 75.9%, že bude mít plat menší nebo rovno 50 tisíc dolarů za rok a zároveň takových lidí je 24 720 v celém datasetu.

### Příklad 2
Zajímavý výsledek dostaneme, pokud sledujeme korelaci atributů `marital-status` a `income`: 

In [10]:
clm = cleverminer(df=adultWithGdp, proc='4ftMiner',
    quantifiers= {'conf':0.95, 'Base':10000},
    ante ={
        'attributes':[
            {'name': 'marital-status', 'type': 'subset', 'minlen': 1, 'maxlen': 1}
        ], 'minlen':1, 'maxlen':1, 'type':'con'},
    succ ={
        'attributes':[
            {'name': 'income', 'type': 'subset', 'minlen': 1, 'maxlen': 1}
        ], 'minlen':1, 'maxlen':1, 'type':'con'}
)

Cleverminer version 1.0.2. Note: This version is for personal and educational use only. If you need PRO version (support, fixing structures for compactibility in future versions for production deployment, additional development, licensing of commercial use of subroutines used), feel free to ask authors. Most of these functionalities are maintained in best-effort, as soon as this project is at given conditions for free use and rapid development is needed, they cannot be guaranteed.
Starting data preparation ...
Encoding columns into bit-form...
Encoding columns into bit-form...done
Data preparation finished.
Will go for  4ftMiner
Starting to mine rules.
Done. Total verifications : 1, rules 1,control number:0, times: prep 3.785076856613159, processing 0.002240896224975586


Jak si z kódu můžeme povšimnout, snažíme se nyní najít vztah mezi manželským statusem a příjmem s velmi vysokou konfidencí - 95%. Otázka tedy nyní zní: Pokud si vezmeme člověka s určitým manželským statusem, je v nějakém případě pravděpodobnost, že daná osoba vydělává více či méně než 50k za rok, větší než 95% a zároveň takových osob je více než 10 000?

Po spuštění zjistíme, že takové pravidlo skutečně existuje:

In [11]:
clm.print_rulelist()


List of rules:
RULEID BASE  CONF  AAD    Rule
     1 10192 0.954 +0.257 marital-status(Never-married) => income(<=50K) | ---



Podíváme se tedy na detaily:

In [12]:
clm.print_rule(1)



Rule id : 1

Base : 10192  Relative base : 0.313  CONF : 0.954  AAD : +0.257  BAD : -0.257

Cedents:
  antecedent : marital-status(Never-married)
  succcedent : income(<=50K)
  condition  : ---

Fourfold table
    |  S  |  ¬S |
----|-----|-----|
 A  |10192|  491|
----|-----|-----|
¬A  |14528| 7350|
----|-----|-----|



Z výsledku můžeme vyvodit následující závěr: Jestliže osoba nikdy nebyla v manželském vztahu, pak je 95% šance, že vydělává méně než 50k za rok a zároveň je takových lidí více než 10000. 

Z tabulky si ale můžeme také povšimnout dalšího pravidla: Pokud si osoba v životě již někoho vzala, pak je 33,5% šance, že má plat vyšší než 50k za rok - vypočteno na kalkulačce jako 7350(počet lidí, kteří si již někoho vzali a vydělávají více než 50k za rok) / (14528 + 7350(Součet všech lidí, kteří si již někoho vzali)). Teoreticky tedy můžeme říci, že začátek manželského vztahu zvyšuje pravděpodobnost, že daná osoba vydělává více, než 50k za rok, o 28.5% -> více než šestinásobně.

V kódu je uvedeno několik dalších příkladů na 4ft miner s popisem v komentářích.

## Krok 4 - [CF Miner](https://lispminer.vse.cz/guhate/doku.php?id=lm_guha_te_cf_proc)

Reference na [Clever Miner](https://www.cleverminer.org/docs-page.html#section-4)

Analytická procedura CF-Miner hledá všechna zajímavá rozdělení frekvencí kategorií předzpracovaného atributu počítaná na podmnožinách analyzovaných dat zadaných pomocí [booleovských atributů s bohatou syntaxí](https://lispminer.vse.cz/wiki/doku.php?id=lmtask:settings:ftcedent).

Analytickou úlohu v CF mineru můžeme definovat například takto:

Existuje velmi výrazně převyšující hodnota atributu `income` pro nějakou kombinaci údajů `marital-status` a `relationship`?

Vzhledem k tomu, že nás zajímá výrazně převažující hodnta, nastavíme kvantifikátor `RelMax` na 0.99. CF miner pak bude hledat kombinaci hodnot atributů `marital-status` a `relationship` tak, aby jedna z hodnot byla obsažena minimálně v 99% případů. Pomocí kvantifikátoru `Base` specifikujeme počet záznamů vyhovující podmínkám. Reference na další kvantifikátory lze nalézt v dokumentaci [Clever mineru](https://www.cleverminer.org/docs-page.html#section-4) v sekci CF miner.

Ve slovníku cond si dále můžeme povšimnout dvou již zmíněných atributů. Syntax je zde stejný jako u 4ft-mineru. Povšimněme si ale nyní definice parametrů `minlen` a `maxlen` za polem atributů. `maxlen` je v tomto případě třeba nastavit na 2, aby vyhledával CF miner vztah mezi `income` a <b>kombinací</b> atributů `marital-status` a `relationship`.

In [18]:
clm = cleverminer(df=adultWithGdp,target='income',proc='CFMiner',
    quantifiers= {'Base': 4000, 'RelMax': 0.99},
    cond = {
        'attributes':[
            {'name': 'marital-status', 'type': 'subset', 'minlen': 1, 'maxlen': 1},
            {'name': 'relationship', 'type': 'subset', 'minlen': 1, 'maxlen': 1}
        ], 'minlen':1, 'maxlen':2, 'type':'con'}
)

Cleverminer version 1.0.2. Note: This version is for personal and educational use only. If you need PRO version (support, fixing structures for compactibility in future versions for production deployment, additional development, licensing of commercial use of subroutines used), feel free to ask authors. Most of these functionalities are maintained in best-effort, as soon as this project is at given conditions for free use and rapid development is needed, they cannot be guaranteed.
Starting data preparation ...
Encoding columns into bit-form...
Encoding columns into bit-form...done
Data preparation finished.
Will go for  CFMiner
Starting to mine rules.
Done. Total verifications : 9, rules 1,control number:0, times: prep 3.945718765258789, processing 0.005689144134521484


Po spuštění tohoto kódu vypíšeme pravidlo, které CF miner našel:

In [14]:
clm.print_rule(1)



Rule id : 1

Base :  4485  Relative base : 0.138  Steps UP (consecutive) :     0  Steps DOWN (consecutive) :     1  Steps UP (any) :     0  Steps DOWN (any) :     1  Histogram maximum :  4451  Histogram minimum :    34  Histogram relative maximum : 0.992 Histogram relative minimum : 0.008

Condition  : marital-status(Never-married) & relationship(Own-child)

Histogram [4451, 34]



Vidíme, že CF miner skutečně takový vztah našel, dokážeme vyčíst, že pokud má člověk dítě a nikdy nebyl v manželství, pak má 99% šanci, že vydělává méně než 50k za rok a zároveň takových lidí je v datasetu minimálně 4400. V histogramu vidíme: [4451, 34] - pouze 34 takových lidí z celkových 4485 vydělává více než 50k za rok.

Více příkladů s CF-minerem sem bude přidáno. Některé se již nachází v kódu.

## Krok 5 - [SD4ft-Miner](https://lispminer.vse.cz/guhate/doku.php?id=lm_guha_te_sd4ft_proc)

Reference na [Clever Miner](https://www.cleverminer.org/docs-page.html#section-4)

Procedura  SD4ft-Miner hledá rozdíly v konfidenci různých podmnožin dat. Podobně jako 4ft-Miner přijímá `succedent` a `antecedent`, ale k tomu navíc ještě dva booleovské atributy které určují podmnožiny dat. Jako quantifier se pak volí z následujících: base hodnoty pro množiny (počet záznamů) absolutní a relativní, confidence pro množiny a relativní či absolutní rozdíl v konfidencích.

### Příklad 1 (SD4ft)

Pojďme se podívat na praktický příklad. Chceme zjistit jestli existují dvě takové země, že v jedné je alespoň dvakrát taková šance, že úroveň dosaženého vzdělání ovlivňuje výši příjmů a v každé množině máme base 100. Formálnějším zápisem by to vypadalo takto: $Country(?) \times Country(?): [Education \approx Income]$

In [34]:
clm = cleverminer(df=adultWithGdp, proc='SD4ftMiner',
               quantifiers= {'Base1':100, 'Base2':100, 'Ratioconf' : 2.0},
               ante ={
                    'attributes':[
                        {'name': 'education', 'type': 'subset', 'minlen': 1, 'maxlen': 2}
                    ], 'minlen':1, 'maxlen':1, 'type':'con'},
               succ ={
                    'attributes':[
                        {'name': 'income', 'type': 'subset', 'minlen': 1, 'maxlen': 1}
                    ], 'minlen':1, 'maxlen':1, 'type':'con'},
               frst ={
                    'attributes':[
                        {'name': 'country', 'type': 'subset', 'minlen': 1, 'maxlen': 1}
                    ], 'minlen':1, 'maxlen':1, 'type':'con'},
               scnd ={
                    'attributes':[
                        {'name': 'country', 'type': 'subset', 'minlen': 1, 'maxlen': 1}
                    ], 'minlen':1, 'maxlen':1, 'type':'con'}
               )

Cleverminer version 1.0.2. Note: This version is for personal and educational use only. If you need PRO version (support, fixing structures for compactibility in future versions for production deployment, additional development, licensing of commercial use of subroutines used), feel free to ask authors. Most of these functionalities are maintained in best-effort, as soon as this project is at given conditions for free use and rapid development is needed, they cannot be guaranteed.
Starting data preparation ...
Encoding columns into bit-form...
Encoding columns into bit-form...done
Data preparation finished.
Will go for  SD4ftMiner
Starting to mine rules.
Done. Total verifications : 419832, rules 3,control number:0, times: prep 4.082936763763428, processing 369.00958490371704


In [36]:
clm.print_rule(1)



Rule id : 1

Base1 :   145 Base2 :   171  Relative base 1 : 0.004 Relative base 2 : 0.005 CONF1 : 0.973  CONF2 : +0.402  Delta Conf : +0.571 Ratio Conf : +2.419

Cedents:
  antecedent : education(5th-6th Doctorate)
  succcedent : income(<=50K)
  condition  : ---
  first set  : country(Mexico)
  second set : country(United-States)

Fourfold tables:
FRST|  S  |  ¬S |  SCND|  S  |  ¬S |
----|-----|-----|  ----|-----|-----| 
 A  |  145|    4|   A  |  171|  254|
----|-----|-----|  ----|-----|-----|
¬A  |  465|   29|  ¬A  |21828| 6917|
----|-----|-----|  ----|-----|-----|



Vidíme, že procedura našla právě tři taková pravidla pro která tyto omezení sedí. My se budeme dále zabívat pouze prvním, protože další dvě jsou v podstatě ten samý výsledek, jen jiná úrověň vzdělání. Jak teda na výsledku vidíme, procedura našla následující pravidlo: $Country(Mexico) \times Country(US): Education(5th-6th Doctorate) \Rightarrow _{2,100,100} Income(<=50K)$

Pojďme se podívat na výsledek od konce, co nám řikají 4ft tabulky? Prví říká, že pokud osoba bydlí v Mexiku a dosáhne vzdělání na úrovni _5th-6th Doctorate_, pak je 97% šance, že bude mít příjem _<=50K_ a zároveň je takových osob alespoň 100. Druhá říká podobné tvrzení, jen že pokud osoba žije v US a dosáhne toho samého vzdělání, tak šance že bude mít roční plat _<=50K_ je 40%. A celkový výsledek na to co jsme se tázali je kombinace těchto dvou poznatků, tedy pokud osoba žije v Mexiku a dosáhne vzdělání na úrovni _5th-6th Doctorate_ pak má více než dvojnásobnou (presněji je to $2.4\times$) pravděpodobnost, že bude mít plat _<=50K_ oproti osobě žijící v US s tím samým vzděláním.

In [37]:
clm.print_rule(2)
clm.print_rule(3)



Rule id : 2

Base1 :   147 Base2 :   753  Relative base 1 : 0.005 Relative base 2 : 0.023 CONF1 : 0.961  CONF2 : +0.464  Delta Conf : +0.497 Ratio Conf : +2.072

Cedents:
  antecedent : education(5th-6th Masters)
  succcedent : income(<=50K)
  condition  : ---
  first set  : country(Mexico)
  second set : country(United-States)

Fourfold tables:
FRST|  S  |  ¬S |  SCND|  S  |  ¬S |
----|-----|-----|  ----|-----|-----| 
 A  |  147|    6|   A  |  753|  871|
----|-----|-----|  ----|-----|-----|
¬A  |  463|   27|  ¬A  |21246| 6300|
----|-----|-----|  ----|-----|-----|



Rule id : 3

Base1 :   146 Base2 :   220  Relative base 1 : 0.004 Relative base 2 : 0.007 CONF1 : 0.973  CONF2 : +0.367  Delta Conf : +0.606 Ratio Conf : +2.650

Cedents:
  antecedent : education(5th-6th Prof-school)
  succcedent : income(<=50K)
  condition  : ---
  first set  : country(Mexico)
  second set : country(United-States)

Fourfold tables:
FRST|  S  |  ¬S |  SCND|  S  |  ¬S |
----|-----|-----|  ----|-----|-----

Pro zajímavost se podíváme ještě na ty další dva výsledky. Jak vidíme, jde o ty samé země, ten samý plat, jediné co se mění je úroveň vzdělání (místo _5th-6th Doctorate_ to je _5th-6th Masters_ a _5th-6th Prof-school_) a samozřejmě se tím liší i konfidence (96% v Mexiku, 46% v US a 97% v Mexiku, 37% v US) a díky tomu se liší i násobnost ($2\times$ a $2.65\times$). Formální zápis pravidel je obdobný prvnímu pravidlu, opět se mění pouze úroveň vzdělání.

### Příklad 2 (SD4ft)

Zkusíme si ještě jeden příklad. Jelikož SD4ft-Miner je velmi podobný 4ft-Mineru, tak si zkusíme podobné cvičení jako v Příkladu 1. Pro zopakování, zde je intepratace dotazu v srozumitelném jazyce:
> Jestliže si vezmeme pro osobu jednu nebo dvě hodnoty z `AgeStatus`(Old/Yound/Middle aged), je v nějakém případě pravděpodobnost, že daná osoba bude vydělávat více či méně než 50K, vyšší než 70% a zároveň je takových osob v datasetu více než 18 000?

Abychom to dali do kontextu SD4ft-Mineru, musíme dotaz trochu upravit. Zaprvé snížíme base, protože hodně zemí v našem datasetu nemá 18000 záznamů a v kontextu porovnání by nás to pak velmi limitovalo. Snížíme base opět na 100. Dále, původní dotaz se ptá na pravděpodobnost vyšší než 70%, my se ale v SD4ft=Mineru neptáme na absolutní pravděpodobnost (i když tu informaci máme potom k dispozici), ale ptáme se na rozdíl pravděpodobnosti mezi podmnožinami. Finální dotaz tedy bude:

Jsou dvě takové země, že v jedné je alespoň o 30% vetší šance že věk osoby dané země ovlivňuje příjem tázané osoby? Formálně pak podobně jako v předchozém příkladu" $Country(?) \times Country(?): [AgeStatus \approx Income]$

In [42]:
clm = cleverminer(df=adultWithGdp, proc='SD4ftMiner',
               quantifiers= {'Base1':100, 'Base2':100, 'Ratioconf' : 1.3},
               ante ={
                    'attributes':[
                        {'name': 'AgeStatus', 'type': 'subset', 'minlen': 1, 'maxlen': 2}
                    ], 'minlen':1, 'maxlen':1, 'type':'con'},
               succ ={
                    'attributes':[
                        {'name': 'income', 'type': 'subset', 'minlen': 1, 'maxlen': 1}
                    ], 'minlen':1, 'maxlen':1, 'type':'con'},
               frst ={
                    'attributes':[
                        {'name': 'country', 'type': 'subset', 'minlen': 1, 'maxlen': 1}
                    ], 'minlen':1, 'maxlen':1, 'type':'con'},
               scnd ={
                    'attributes':[
                        {'name': 'country', 'type': 'subset', 'minlen': 1, 'maxlen': 1}
                    ], 'minlen':1, 'maxlen':1, 'type':'con'}
               )

Cleverminer version 1.0.2. Note: This version is for personal and educational use only. If you need PRO version (support, fixing structures for compactibility in future versions for production deployment, additional development, licensing of commercial use of subroutines used), feel free to ask authors. Most of these functionalities are maintained in best-effort, as soon as this project is at given conditions for free use and rapid development is needed, they cannot be guaranteed.
Starting data preparation ...
Encoding columns into bit-form...
Encoding columns into bit-form...done
Data preparation finished.
Will go for  SD4ftMiner
Starting to mine rules.
Done. Total verifications : 10584, rules 3,control number:0, times: prep 4.199059009552002, processing 10.384336233139038


Jak vidíme, Clever Miner nám našel právě tři taková pravidla, rozebereme si je od konce.

In [53]:
clm.print_rule(3)



Rule id : 3

Base1 :   326 Base2 : 13795  Relative base 1 : 0.010 Relative base 2 : 0.424 CONF1 : 0.916  CONF2 : +0.673  Delta Conf : +0.243 Ratio Conf : +1.361

Cedents:
  antecedent : AgeStatus(Middle aged)
  succcedent : income(<=50K)
  condition  : ---
  first set  : country(Mexico)
  second set : country(United-States)

Fourfold tables:
FRST|  S  |  ¬S |  SCND|  S  |  ¬S |
----|-----|-----|  ----|-----|-----| 
 A  |  326|   30|   A  |13795| 6707|
----|-----|-----|  ----|-----|-----|
¬A  |  284|    3|  ¬A  | 8204|  464|
----|-----|-----|  ----|-----|-----|



Jak můžeme vidět, jedná se opět o země US a Mexiko, tentokrát osoba žijící v Mexiku má o 30% větší šanci, že pokud spadá do věkové skupiny _Middle aged_, pak bude vydělávat méně než 50K ročně. Jdeme na další výsledek.

In [54]:
clm.print_rule(2)



Rule id : 2

Base1 :   610 Base2 :   137  Relative base 1 : 0.019 Relative base 2 : 0.004 CONF1 : 0.949  CONF2 : +0.692  Delta Conf : +0.257 Ratio Conf : +1.371

Cedents:
  antecedent : AgeStatus(Middle aged Young)
  succcedent : income(<=50K)
  condition  : ---
  first set  : country(Mexico)
  second set : country(Philippines)

Fourfold tables:
FRST|  S  |  ¬S |  SCND|  S  |  ¬S |
----|-----|-----|  ----|-----|-----| 
 A  |  610|   33|   A  |  137|   61|
----|-----|-----|  ----|-----|-----|
¬A  |    0|    0|  ¬A  |    0|    0|
----|-----|-----|  ----|-----|-----|



Dříve než se pustíme bezhlavě do interpretace výsledků je třeba si všimnout jednoho faktu, a to že obě 4ft tabulky mají prázdný poslední řádek. To znamená, že ani v Mexiku ani na Filipínách není člověk který by **ne**zapadal do kategorií _Middle aged_ nebo _Young_ (tedy když si to obrátíme, není tam člověk ve skupině _Old_). To ale logicky nedává smysl, určitě v těch zemích žijí lidé starší 60ti let. Máme tedy rozpor mezi realitou a datasetem, a díky tomuto rozporu nemůžeme brát výsledek jako směrodatný. Když data nereflektují realitu, pak ani predikce nemůže správně předvídat realitu. Výsledek 2 tedy označíme jako statisticky nezajímavý.

In [55]:
clm.print_rule(1)



Rule id : 1

Base1 :   326 Base2 :   302  Relative base 1 : 0.010 Relative base 2 : 0.009 CONF1 : 0.916  CONF2 : +0.686  Delta Conf : +0.229 Ratio Conf : +1.334

Cedents:
  antecedent : AgeStatus(Middle aged)
  succcedent : income(<=50K)
  condition  : ---
  first set  : country(Mexico)
  second set : country(?)

Fourfold tables:
FRST|  S  |  ¬S |  SCND|  S  |  ¬S |
----|-----|-----|  ----|-----|-----| 
 A  |  326|   30|   A  |  302|  138|
----|-----|-----|  ----|-----|-----|
¬A  |  284|    3|  ¬A  |  135|    8|
----|-----|-----|  ----|-----|-----|



A v posledním (technicky vzato prvním) výsledku vidíme, že druhá množina je "variabilní". Výsledek lze pak interpretovat následujícím způsobem:

Pokud osoba spadá do kategorie _Middle aged_ a žije v Mexiku, pak má alespoň o 30% větší šanci, že bude mít příjem _<=50K_ oproti osobě té samé věkové kategorie žijící mimo Mexiko.